In [1]:
import os

os.chdir("..")
print(f"Changed working directory to: {os.getcwd()}")

Changed working directory to: /home/jovyan/work/FlareSense


In [2]:
import torch
import mlflow
import dagshub
import torchmetrics
import src.utils.data as data
import pytorch_lightning as pl

from torchvision import transforms
from tqdm.notebook import tqdm
from src.models.ResNet50BinaryClassifier import ResNet50BinaryClassifier

mlflow.pytorch.autolog()
torch.set_float32_matmul_precision("high")

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_config.py:318: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
2023/11/22 15:50:31 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
2023-11-22 15:50:31.095709: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI 

## Training

Defining the data and model

In [3]:
model = ResNet50BinaryClassifier(lr=1e-4, weight_decay=1e-2)
data_folder_path = "data/raw/burst_images/"

data_module = data.ECallistoDataModule(
    data_folder=data_folder_path,
    transform=transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize((193, 240), antialias=True),
            transforms.ToTensor(),
        ]
    ),
    batch_size=64,
    num_workers=10,
    val_ratio=0.15,
    test_ratio=0.15,
    split_by_date=True,
    noburst_to_burst_ratio=4,
    filter_instruments=["australia_assa_02"],
)
data_module.setup()

Not enough test bursts (81), minimum 119 needed
Reshuffling...

Too many validation bursts (350), maximum 266 allowed
Reshuffling...

Dataset split successfully
Train:		890 bursts
Validation:	146 bursts
Test:		147 bursts


Training and logging the model

In [4]:
dagshub.init("FlareSense", "FlareSense", mlflow=True)
mlflow.start_run()

mlflow.log_params({
    "model": "ResNet50",
    "batch_size": data_module.batch_size,               
    "val_ratio": data_module.val_ratio,
    "test_ratio": data_module.test_ratio,
    "min_factor_val_test": data_module.min_factor_val_test,
    "max_factor_val_test": data_module.max_factor_val_test,
    "noburst_to_burst_ratio": data_module.noburst_to_burst_ratio,
    "split_by_date": data_module.split_by_date,
    "filter_instruments": data_module.filter_instruments,
})

run_id = mlflow.active_run().info.run_id
print(f"Run ID: {run_id}")
print(f"Link: https://dagshub.com/FlareSense/FlareSense/experiments/#/experiment/m_{run_id}")

trainer = pl.Trainer(max_epochs=50, log_every_n_steps=1)

trainer.fit(
    model,
    train_dataloaders=data_module.train_dataloader(),
    val_dataloaders=data_module.val_dataloader(),
)

trainer.test(model, dataloaders=data_module.test_dataloader())

mlflow.end_run()

Repository initialized!

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Run ID: 7063e6c77e824e1288bdc8156967fcd9
Link: https://dagshub.com/FlareSense/FlareSense/experiments/#/experiment/m_7063e6c77e824e1288bdc8156967fcd9


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type            | Params
----------------------------------------------
0 | resnet50  | ResNet          | 23.5 M
1 | precision | BinaryPrecision | 0     
2 | recall    | BinaryRecall    | 0     
----------------------------------------------
22.1 M    Trainable params
1.4 M     Non-trainable params
23.5 M    Total params
94.040    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
2023/11/22 15:59:00 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.07608781009912491    │
│      test_precision       │    0.9318181872367859     │
│        test_recall        │    0.8367347121238708     │
└───────────────────────────┴───────────────────────────┘

## Testing

Downloading the model from mlflow and putting it on the correct device

In [5]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

mlflow.set_tracking_uri('https://dagshub.com/FlareSense/Flaresense.mlflow')
model = mlflow.pytorch.load_model(f"runs:/{run_id}/model/").to(device)
model.eval()

ResNet50BinaryClassifier(
  (resnet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequ

Run through testdata

In [6]:
test_labels_list = []
test_preds_list = []
with torch.no_grad():
    for batch in tqdm(data_module.test_dataloader()):
        images, info = batch
        images = images.to(device)
        binary_labels = [0 if label == "no_burst" else 1 for label in info['label']]
        binary_labels = torch.tensor(binary_labels).int().view(-1, 1)
        binary_labels = binary_labels.to(device)
        
        outputs = model(images.expand(-1, 3, -1, -1))
        predictions = (outputs >= 0.5).int()
        
        test_labels_list.append(binary_labels)
        test_preds_list.append(predictions)

  0%|          | 0/57 [00:00<?, ?it/s]

Create confmatrix

In [7]:
test_labels = torch.cat(test_labels_list, dim=0)
test_preds = torch.cat(test_preds_list, dim=0)

confmat_metric = torchmetrics.ConfusionMatrix(num_classes=2, task="binary").to(device)
confmat_metric(test_preds, test_labels)

tensor([[3459,    9],
        [  24,  123]], device='cuda:0')